In [221]:

from sympy.solvers import solve
from sympy import Symbol
import torch as t
import math
import numpy as np
from scipy import optimize

# initial guess:
x1 = Symbol('x1')
x2 = 0
x1 = solve(x1**2/4 + x2**2/5 + (x1+x2)**2/25 - 1, x1)[1]
x3 = x1+x2
print(x2,x3)
x = t.tensor([10*math.sqrt(29)/29, 0, 10*math.sqrt(29)/29])
print(x)

# define problem:
def obj(x):
    return x[0]**2+x[1]**2+x[2]**2

def h(x):
    return t.tensor([[x[0]**2/4+x[1]**2/5+x[2]**2/25-1],
                    [x[0]+x[1]-x[2]]])

# define d and s, Choice d = x1 (x[0]); s = [x2 x3] (x[1], x[2])
def d(x):
    return t.tensor([x[0]])
def s(x):
    return t.tensor([x[1],x[2]])


# Calculate Gradient: e means partial derivative symbol
def ef_ed(x):
    return t.tensor([2*x[0]])

def ef_es(x):
    return t.tensor([2*x[1],2*x[2]])

def eh_es(x):
    return t.tensor([[2*x[1]/5,2*x[2]/25],
                    [1,-1]])

def eh_ed(x):
    return t.tensor([[x[0]/2],
                     [1]])

def df_dd(x):
    return ef_ed(x) - t.matmul(t.matmul(ef_es(x),t.inverse(eh_es(x))),eh_ed(x))

def linesearch(x):
    a = 1
    b = 0.5
    c = 0.3
    i = 0
    def f_a(x):
        x_k_1 = t.cat([d(x)-a*df_dd(x), t.squeeze(s(x) + a*t.transpose(t.matmul(t.inverse(eh_es(x)),eh_ed(x))*df_dd(x),0,1))])
        return obj(x_k_1)
    def f__a(x):
        return obj(x) - a*c*df_dd(x)**2
    while(f_a(x)>f__a(x)):
        a = b*a
        i = i+1
        if i>=50:
            print('linesearch issue')
            exit()
    return a

def solver(x):
    j = 0
    while t.norm(h(x))>0.06:
        Ch = t.matmul(t.inverse(t.add(t.matmul(eh_es(x),t.transpose(eh_es(x),0,1)),t.eye(2))),t.transpose(eh_es(x),0,1))
        s_k_1 = t.squeeze(s(x) - t.transpose(t.matmul(Ch,h(x)),0,1))
        x = t.cat([d_k_1, s_k_1])
        j = j+1
    return x

E = 0.01
while df_dd(x)**2>E:
    a = linesearch(x)
    d_k_1 = d(x) - a*df_dd(x)
    s0_k_1 = t.squeeze(s(x) + a*t.transpose(t.matmul(t.inverse(eh_es(x)),eh_ed(x))*df_dd(x),0,1))
    x = t.cat([d_k_1, s0_k_1])
    x = solver(x)
print(x)
print(h(x))
print(obj(x))


0 10*sqrt(29)/29
tensor([1.8570, 0.0000, 1.8570])
tensor([ 1.9331, -0.4039,  1.5202])
tensor([[0.0593],
        [0.0091]])
tensor(6.2109)
tensor([ 1.9166, -0.6012,  1.3029])
tensor([[0.0585],
        [0.0125]])
tensor(5.7323)
tensor([ 1.8487, -0.9179,  0.9131])
tensor([[0.0563],
        [0.0176]])
tensor(5.0940)
tensor([ 1.7629, -1.1450,  0.5972])
tensor([[0.0534],
        [0.0207]])
tensor(4.7753)
tensor([ 1.6929, -1.2839,  0.3864])
tensor([[0.0521],
        [0.0227]])
tensor(4.6636)
tensor([ 1.6649, -1.3347,  0.3075])
tensor([[0.0531],
        [0.0227]])
tensor(4.6481)
tensor([ 1.6345, -1.3860,  0.2258])
tensor([[0.0541],
        [0.0227]])
tensor(4.6435)
tensor([ 1.6278, -1.3964,  0.2087])
tensor([[0.0542],
        [0.0227]])
tensor(4.6432)


In [222]:
from scipy.optimize import minimize
fun = lambda x: x[0]**2 + x[1]**2+x[2]**2

cons = ({'type': 'eq', 'fun': lambda x:  x[0]**2/4+x[1]**2/5+x[2]**2/25-1},
        {'type': 'eq', 'fun': lambda x:  x[0]+x[1]-x[2]})
bnds = ((-2, 2), (-2.3, 2.3), (-5, 5))

x0 = [1,0,0]
res = minimize(fun, x0, method='SLSQP',bounds=bnds,
               constraints=cons)
res

     fun: 4.411764720309136
     jac: array([ 3.14753878, -2.75414187,  0.39339691])
 message: 'Optimization terminated successfully'
    nfev: 24
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([ 1.57376938, -1.37707095,  0.19669843])